In [1]:
import numpy as np
import numpy.ma as ma  # masked array
from numpy import random

from scipy.signal import (cwt, ricker, find_peaks, find_peaks_cwt, peak_widths)
from scipy.ndimage import label, generate_binary_structure, find_objects, binary_dilation  # 2020-09-03

from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh import colors
from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap

In [2]:
default_plot_width =1200
default_plot_height = 900

In [3]:
maxX = 500
maxY = 300

nri = 60
lengs = random.randint(100, size=nri)

walk = {
    0: ( 0, -1),
    1: ( 1, -1),
    2: ( 1,  0),
    3: ( 1,  1),
    4: ( 0,  1),
}

ris = []
for i in range(nri):
    ri = np.asarray([random.randint(maxX), random.randint(maxY)])
    ris.append(ri)
    for j in range(lengs[i]):
        ri = ri + walk[random.randint(5)]
        if (ri[0] in range(0,maxX)) and (ri[1] in range(0,maxY)):
            ris.append(ri)
ris = np.asarray(ris)

In [4]:
# 2020-09-02 Caminho aleatório
fig1 = figure(
   title = 'Caminhos aleatórios',
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   x_axis_label='channels', y_axis_label='counts'
)
fig1.circle(ris[:,0], ris[:,1], legend_label="cwt peaks", color="navy",size=2)
show(fig1)

In [5]:
# Agora, criacao e processamento da imagem:

In [6]:
ima1 = np.zeros((maxX,maxY))

In [7]:
np.shape(ima1)

(500, 300)

In [8]:
for ri in ris:
    ima1[ri[0],ri[1]]=1
ima1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
# 2020-09-02 Refazendo o plot com os objetos identificados com stru22 (=adjacentes e na diagonal imediata)

In [10]:
stru22 = generate_binary_structure(2,2)
stru22

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [11]:
lblimag22, num_feat_imag22 = label(ima1,stru22)
lblimag22, num_feat_imag22
objs = find_objects(lblimag22)
ridges_rectangles = [lblimag22[loc] for loc in objs]
alltops = [i[1].stop-0.5 for i in objs]
allbottoms = [i[1].start-0.5 for i in objs]
alllefts = [i[0].start-0.5 for i in objs]
allrights = [i[0].stop-0.5 for i in objs]
len(objs)

58

In [12]:
# https://stackoverflow.com/questions/3941517/converting-list-to-args-when-calling-function#3941529
# 2020-09-02 Python: * faz função ver lista comum como lista de argumentos
randcolors = [[colors.RGB(*random.randint(16,240,3))] for i in range(len(objs))]

In [13]:
# 2020-09-02 Caminhos aleatórios identificados (porém, com superposição)
fig2 = figure(
   title='Figura 2: Agrupamentos identificados',
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   x_axis_label='channels', y_axis_label='counts'
)
fig2.circle(ris[:,0], ris[:,1], legend_label="positions", color="navy",size=2)
fig2.quad(top=alltops, bottom=allbottoms, left=alllefts, right=allrights, color=randcolors, fill_alpha=0.5)
show(fig2)

In [14]:
# 2020-09-03 Refazendo o plot com os objetos identificados com stru20 (=só adjacentes)

In [15]:
stru20 = generate_binary_structure(2,0)
stru20

array([[False,  True, False],
       [ True,  True,  True],
       [False,  True, False]])

In [16]:
lblimag20, num_feat_imag20 = label(ima1,stru20)
lblimag20, num_feat_imag20
objs = find_objects(lblimag20)
ridges_rectangles = [lblimag20[loc] for loc in objs]
alltops = [i[1].stop-0.5 for i in objs]
allbottoms = [i[1].start-0.5 for i in objs]
alllefts = [i[0].start-0.5 for i in objs]
allrights = [i[0].stop-0.5 for i in objs]
len(objs)

813

In [17]:
# https://stackoverflow.com/questions/3941517/converting-list-to-args-when-calling-function#3941529
# 2020-09-02 Python: * faz função ver lista comum como lista de argumentos
randcolors = [[colors.RGB(*random.randint(16,240,3))] for i in range(len(objs))]

In [18]:
# 2020-09-02 Caminhos aleatórios identificados (porém, com superposição)
fig3 = figure(
   title='Figura 3: Agrupamentos identificados',
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   x_axis_label='channels', y_axis_label='counts'
)
fig3.circle(ris[:,0], ris[:,1], legend_label="positions", color="navy",size=2)
fig3.quad(top=alltops, bottom=allbottoms, left=alllefts, right=allrights, color=randcolors, fill_alpha=0.5)
show(fig3)